# Earthquake damage classification

### 24 August 2016 Central Italy earthquake 🇮🇹
### 26 October 2016 Central Italy earthquakes 🇮🇹
### 30 October 2016 Central Italy earthquake 🇮🇹

## 0: Preliminaries

### Import python libraries 

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona
import folium
import rasterio
import rasterio.mask
import rasterstats as rs
import seaborn as sns
from shapely.geometry import Point
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
starttime = time()

### Set the default coordinate reference system

In [ ]:
dst_crs = "EPSG:4326"

### Indicate paths to input files

In [ ]:
event_name = "Central Italy earthquakes"
dpm_raw_file = "DPM/ARIA_DPM_ALOS2_f540_v0.5u_climMax07454_T1H1B0U0_dpmRaw.tif"
dpm_clipped_file = "DPM/ARIA_DPM_ALOS2_f540_v0.5u_climMax07454_T1H1B0U0_dpmRaw_ShakeMapMask.tif"
# building_footprints_file = "Buildings/Buildings.geojson"
damage_labels_file = "DamageLabels/Budhanilkantha_Data.csv"
shakemap_url = "https://earthquake.usgs.gov/product/shakemap/us20002926/atlas/1594162031303/download/"
shakemap_json = shakemap_url + "cont_mmi.json" # Contours
shakemap_zip = shakemap_url + "shape.zip" # Polygons

## 1: Demarcate affected area

### Draw an empty map centered on the event¶

In [ ]:
lat = +28.231
lon = +84.731

bound_n = lat + 3
bound_e = lon + 3
bound_s = lat - 3
bound_w = lon - 3

m = folium.Map(
    location=[lat, lon], 
    zoom_start=9,
    tiles='Stamen Terrain')

folium.CircleMarker(
    location=[lat, lon],
    radius=5,
    tooltip="Epicenter",
    popup=event_name,
    color="IndianRed",
    fill_color="LightCoral",
    fill=True
).add_to(m)

m

### Draw ShakeMap contours for MMI ≥ V

In [ ]:
style_function = lambda x: {
    'color': x["properties"]["color"],
    'weight': x["properties"]["weight"] if x["properties"]["value"] >= 5 else 0
}
tooltip = folium.features.GeoJsonTooltip(fields=["value"])

folium.GeoJson(
    shakemap_json,
    name="ShakeMap Contours",
    style_function=style_function,
    tooltip=tooltip
).add_to(m)

folium.TileLayer('OpenStreetMap').add_to(m)
folium.LayerControl().add_to(m)

m

### Save the MMI V contour as a vector layer

In [ ]:
shakemap_gdf = gpd.read_file(shakemap_json)
shakemapV_gdf = shakemap_gdf[shakemap_gdf.value==5]
shakemapV_gdf.to_file("ShakeMap/MMI_V_Contours.gpkg", driver="GPKG")
shakemapV_gdf.convex_hull.to_file("ShakeMap/MMI_V_Envelope.gpkg", driver="GPKG")

## 2: Load DPM tiles and mask to built areas within MMI≥V

### Read the DPM raster and mask it using the MMI V contour

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

with fiona.open("ShakeMap/MMI_V_Envelope.gpkg", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(dpm_raw_file) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(dpm_clipped_file, "w", **out_meta) as dest:
    dest.write(out_image)

### Explore the distribution of the clipped DPM pixel values by plotting a histogram¶

In [ ]:
from rasterio.plot import show_hist
dpm_src = rasterio.open(dpm_raw_file)
show_hist(
    dpm_src, bins=64, lw=0.0, stacked=False, alpha=0.3,
    histtype='stepfilled', title="Distribution of DPM Raw Pixel Values")

## 3: Load and plot the building inventory files

### Load the building inventory file for Budhanilkantha 

In [ ]:
# bldgs_gdf = gpd.read_file(building_footprints_file)

### Inspect the building inventory file

In [ ]:
# bldgs_gdf.head()

### Drop unnecessary columns

In [ ]:
# bldgs_gdf.drop(columns=["fid", "ObjectType", "PROMJENA"], inplace=True)

## 4: Join the damage labels to the building inventory

### Load the damage labels ("ground truth" / "test labels")

In [ ]:
usecols = ["BuildingID", "AgeOfBuild", "FootprintA", "NumOfStore", "Slope_Grou", "Constr_Typ", "Type_Floor", "Type_Roof", "Build_Posi", "Build_FtPr", "VrtclStrIr", "DMGgrd", "POINT_X", "POINT_Y"]
dmg_labels_df = pd.read_csv(damage_labels_file, usecols=usecols, index_col=0)
dmg_labels_gdf = gpd.GeoDataFrame(
    dmg_labels_df.drop(["POINT_X", "POINT_Y"], axis=1),
    crs=dst_crs,
    geometry=[Point(xy) for xy in zip(dmg_labels_df.POINT_X, dmg_labels_df.POINT_Y)])

### Inspect the damage labels layer contents

In [ ]:
dmg_labels_gdf.head()

In [ ]:
# Clean up field names
dmg_labels_gdf.rename(
    columns={
    "AgeOfBuild": "AGE", 
    "FootprintA": "FOOTPRINT_AREA", 
    "NumOfStore": "NUM_STOREYS", 
    "Slope_Grou": "SLOPE_GROUND", 
    "Constr_Typ": "TYPE_CONSTR", 
    "Type_Floor": "TYPE_FLOOR", 
    "Type_Roof": "TYPE_ROOF", 
    "Build_Posi": "ADJACENCY", 
    "Build_FtPr": "FOOTPRINT_SHAPE", 
    "VrtclStrIr": "IRREGULARITY", 
    "DMGgrd": "DAMAGE_GRADE",
    }, 
    inplace=True)

# Inspect contents of the revised files
dmg_labels_gdf.head()

In [ ]:
# Inspect the damage grade labels column
ax = (dmg_labels_gdf.DAMAGE_GRADE
      .value_counts()
      .sort_index()
      .plot.bar(
          title="Damage Labels"
      ))
for p in ax.patches:
    ax.annotate(
        p.get_height(), (p.get_x()+p.get_width()/2., p.get_height()), 
        ha='center', va='center', 
        xytext=(0, 10), textcoords='offset points')

### Explanation of damage labels
The damage labels are based on the EMS-98 scale. 
Data source: National Society of Earthquake Technology - Nepal (NSET) and Budhanilkantha Municipality, Kathmandu District

![EMS-98 Damage Grades](https://emergency.copernicus.eu/mapping/sites/default/files/images/DamageAssessement_classification.png)

### Inspect the damage grade labels column after relabeling (if any)

In [ ]:
ax = (dmg_labels_gdf.DAMAGE_GRADE
      .value_counts()
      .sort_index()
      .plot.bar(
          title="Damage Labels",
          color=["LimeGreen", "Yellow", "Gold", "Orange", "Coral", "IndianRed"],
      ))
for p in ax.patches:
    ax.annotate(
        p.get_height(), (p.get_x()+p.get_width()/2., p.get_height()), 
        ha='center', va='center', 
        xytext=(0, 10), textcoords='offset points')

#### Note: This is an imbalanced dataset
The number of samples in Damage Grade "0" is roughly two orders of magnitude larger than the number of samples in Damage Grade "3". Most machine learning multiclass classification algorithms assume that all classes have roughly similar numbers of examples.

To circumvent this issue, one potential route is to modify the training set to have similar numbers of examples in each category, for instance by oversampling training examples from the classes with fewer examples.

Another route is to modify the classification algorithm by changing the way learning is performed, preferably biasing more towards those classes that have fewer examples in the training dataset. This is generally called cost-sensitive learning.

In [ ]:
# Inspect the coordinate reference system used by the damage database
dmg_labels_gdf.crs

In [ ]:
# Reproject the damage dataset to the project coordinate reference system
dmg_labels_gdf = dmg_labels_gdf.to_crs(dst_crs)

### Buffer the points in the damage database to expand each point into a circle of 5m radius

In [ ]:
# Buffer each damage label point using a 20 meter buffer zone 
# and replace the point geometry with the new buffered geometry
# The damage database is in the EPSG:3857 CRS, thus the buffer distance is in metres
# Given the coarser resolution of the ALOS DPM, smaller buffer radii
# result in a large number of NaN DPM assignments
dmg_labels_buffer_gdf = dmg_labels_gdf.copy()
dmg_labels_buffer_gdf["geometry"] = dmg_labels_gdf.to_crs("EPSG:3857").buffer(20).to_crs(dst_crs)
dmg_labels_buffer_gdf.head()

### Join the damage labels to the building inventory

In [ ]:
# Building footprints available from OSM for Budhanilkantha are post-earthquake, 
# so they are missing the collapsed buildings; thus in this case it is better
# to proceed without a building footprint dataset. We treat the buffered damage
# label points as buildings
input_gdf = dmg_labels_buffer_gdf

## 5. Join ShakeMap intensity to building inventory

### Join ShakeMap value to the building inventory

In [ ]:
# Build url for the ShakeMap polygons shapefile and perform a spatial join
tic = time()
shakemap_shp = f"zip+{shakemap_zip}!mi.shp"
shakemap_gdf = gpd.read_file(shakemap_shp)
inputs_gdf = gpd.sjoin(input_gdf, shakemap_gdf, how="left", op="within")
print(f"Spatial join completed in {time() - tic:.0f}s")

In [ ]:
# Drop unneeded columns and rename the remaining ones
inputs_gdf.drop(columns=["index_right", "AREA", "PERIMETER", "PGAPOL_", "PGAPOL_ID", "GRID_CODE"], inplace=True)
inputs_gdf.rename(columns={"Area": "AREA", "layer": "BLDG_TYPE", "PARAMVALUE": "MMI"}, inplace=True)

In [ ]:
# Assign buildings with no shaking intensity to a value of MMI=0
inputs_gdf.fillna(value={"MMI": 0}, inplace=True)

In [ ]:
# Create both string and numeric versions of the shaking intensity vector
inputs_strings = inputs_gdf.astype({"MMI": str, "DAMAGE_GRADE":str})
inputs_numeric = inputs_gdf.astype({"MMI": float, "DAMAGE_GRADE":int})

In [ ]:
# Inspect the joined dataset
inputs_numeric.head()

In [ ]:
sns.set_theme(style="ticks")
sns.jointplot(
    data=inputs_numeric, 
    x="MMI", y="DAMAGE_GRADE", 
    marker="+", s=100, 
    marginal_kws=dict(bins=25, fill=False),
)

## 6: Join DPM values to the input dataset

### Find the maximum DPM value falling within each buffered building polygon

In [ ]:
# This step can take several minutes, depending on the sizes
# of the building inventory + damage database and the DPM raster
tic = time()
with rasterio.open(dpm_clipped_file) as dpm_src:
    dpm_data = dpm_src.read(1, masked=True)
    dpm_meta = dpm_src.profile
    
inputs_list = rs.zonal_stats(
    inputs_numeric,
    dpm_data,
    nodata=-999,
    affine=dpm_meta['transform'],
    geojson_out=True,
    copy_properties=True,
    stats="max")

print(f"Zonal stats query completed in {time() - tic:.0f}s")

# View object type
type(inputs_list)

### Create dataframe with input variable values and labels

In [ ]:
inputs_gdf = gpd.GeoDataFrame.from_features(inputs_list)
inputs_gdf.rename(columns={"max": "DPM_MAX"}, inplace=True)

inputs_gdf.head()

In [ ]:
# Plot a histogram of the joined DPM values
# This ignores NaN values, i.e., those buildings for which no DPM value was assigned
dpm_hist = inputs_gdf.DPM_MAX.hist(bins=20)

In [ ]:
# Assign buildings with no DPM value to a value of DPM=0
inputs_gdf.fillna(value={"DPM_MAX": 0}, inplace=True)
dpm_hist = inputs_gdf.DPM_MAX.hist(bins=20)

## 7: Extract and explore the training features and training labels

### Extract the training labels and values as vectors

In [ ]:
train_values = inputs_gdf[["AGE", "FOOTPRINT_AREA", "NUM_STOREYS", "SLOPE_GROUND", "TYPE_CONSTR", "TYPE_FLOOR", "TYPE_ROOF", "ADJACENCY", "FOOTPRINT_SHAPE", "IRREGULARITY", "MMI", "DPM_MAX"]]
train_labels = inputs_gdf[["DAMAGE_GRADE"]]

In [ ]:
train_values.dtypes

In [ ]:
train_labels.dtypes

### Explore the training data

In [ ]:
ax = (train_labels.DAMAGE_GRADE
      .value_counts()
      .sort_index()
      .plot.bar(
          title="Number of Buildings in Each Damage Grade",
          color=["LimeGreen", "Yellow", "Gold", "Orange", "Coral", "IndianRed"],
      ))
for p in ax.patches:
    ax.annotate(
        p.get_height(), (p.get_x()+p.get_width()/2., p.get_height()), 
        ha='center', va='center', 
        xytext=(0, 10), textcoords='offset points')

### Select subset of features to use for training

In [ ]:
selected_features = ["AGE", "NUM_STOREYS", "SLOPE_GROUND", "TYPE_CONSTR", "TYPE_FLOOR", "TYPE_ROOF", "ADJACENCY", "FOOTPRINT_SHAPE", "IRREGULARITY", "MMI", "DPM_MAX"]
train_values_subset = train_values[selected_features]

### Explore the relationships between the numeric features and labels

In [ ]:
sns.set_theme(style="ticks")
train_values_subset_plot = train_values_subset.replace(0, np.nan)
sns.pairplot(
    train_values_subset_plot.join(train_labels),
    hue="DAMAGE_GRADE",
    hue_order=[5, 4, 3, 2, 1, 0],
    palette={0:"LimeGreen", 1:"Yellow", 2:"Gold", 3:"Orange", 4:"Coral", 5:"IndianRed"},
    markers=["1", "2", "3", "4", "+", "x"],
    kind="scatter",
    diag_kind="hist",
    corner=True,
    plot_kws={'alpha':0.6, 'linewidth':1},
    diag_kws={'fill':True},
    grid_kws={'diag_sharey':False},
)
plt.savefig("Figures/PairPlot_DS5.png")

In [ ]:
# Ignore Damage Grade 0 for the plots
sns.pairplot(
    train_values_subset_plot.join(train_labels).replace(0, np.nan),
    hue="DAMAGE_GRADE",
    hue_order=[5, 4, 3, 2, 1],
    palette={1:"Yellow", 2:"Gold", 3:"Orange", 4:"Coral", 5:"IndianRed"},
    markers=["2", "3", "4", "+", "x"],
    kind="scatter",
    diag_kind="hist",
    corner=True,
    plot_kws={'alpha':0.6, 'linewidth':1},
    diag_kws={'fill':True},
    grid_kws={'diag_sharey':False},
)
plt.savefig("Figures/PairPlot_DS5.png")

## 8: Build the ML model(s)

### Import modules for the machine learning training component

In [ ]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# for unbalanced datasets
from sklearn.utils.class_weight import compute_sample_weight

# for splitting the data into training and test sets
from sklearn.model_selection import train_test_split

# the model(s)
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

# for combining the preprocess with model training
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

### Split dataset into training and test subsets

When performing a (supervised) machine learning experiment, it is common to hold out part of the available data as a test set `X_test, y_test`. The best parameters can be determined by grid search techniques.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_values_subset, train_labels, train_size=0.70, test_size=0.30, random_state=40)

In [ ]:
n_categorical_features = (X_train.dtypes == 'object').sum()
n_numerical_features = ((X_train.dtypes == 'float') | (X_train.dtypes == 'int')).sum()
print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of training features: {X_train.shape[1]}")
print(f"Number of categorical features: {n_categorical_features}")
print(f"Number of numerical features: {n_numerical_features}")

We will create a HistGradientBoostingRegressor estimator that natively handles categorical features. We let the estimator know which features are categorical.

First, we create an ordinal pipeline that will treat categorical features as if they were ordered quantities, i.e. the categories will be encoded as 0, 1, 2, etc.

In [ ]:
ordinal_encoder = make_column_transformer(
    (OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan),
     make_column_selector(dtype_include='object')),
    remainder='passthrough')

In [ ]:
# The ordinal encoder will first output the categorical features, and then the
# continuous (passed-through) features
categorical_mask = ([True] * n_categorical_features + [False] * n_numerical_features)

In [ ]:
sample_weight = compute_sample_weight(class_weight="balanced", y=y_train)

### Use a Histogram Gradient Boosting Classifier

In [ ]:
pipe = make_pipeline(
    ordinal_encoder,
    HistGradientBoostingClassifier(
        random_state=42,
        categorical_features=categorical_mask)
)
pipe

### Train a HistGradientBoostingClassifier model with scikit-learn defaults for all parameters

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

In [ ]:
tic = time()
gs = pipe
gs.fit(X_train, y_train.values.ravel(), histgradientboostingclassifier__sample_weight=sample_weight)

In [ ]:
print(f"Model fit completed in {time() - tic:.0f}s")

### Evaluate the model prediction performance on the _training set_

In [ ]:
# Evaluation metrics
# from sklearn.metrics import roc_curve # restricted to the binary classification case
# from sklearn.metrics import plot_precision_recall_curve # restricted to the binary classification case
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

#### Compute the F1 score, also known as balanced F-score or F-measure for the _training_ set

The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:

`F1 = 2 * (precision * recall) / (precision + recall)`

In our multi-class case, this is the average of the F1 score of each class with weighting depending on the average parameter.

In [ ]:
# Calculate metrics globally by counting the total true positives, 
# false negatives and false positives.
in_sample_preds = gs.predict(train_values_subset)
f1_score(train_labels, in_sample_preds, average='micro')

In [ ]:
# Calculate metrics for each label, and find their average weighted 
# the number of true instances for each label. This alters ‘macro’ to 
# account for label imbalance; it can result in an F-score that is not 
# between precision and recall.
f1_score(train_labels, in_sample_preds, average='weighted')

In [ ]:
# Calculate metrics for each label, and find their unweighted mean. 
# This does not take label imbalance into account.
f1_score(train_labels, in_sample_preds, average='macro')

#### Print balanced accuracy score for predictions on the _training_ set

The balanced_accuracy_score function computes the balanced accuracy, which avoids inflated performance estimates on imbalanced datasets. It is the macro-average of recall scores per class or, equivalently, raw accuracy where each sample is weighted according to the inverse prevalence of its true class. For balanced datasets, the score is equal to accuracy.

In [ ]:
balanced_accuracy_score(train_labels, in_sample_preds)

#### Print classification report for predictions on the _training_ set
The precision is the ratio `tp / (tp + fp)` where `tp` is the number of true positives and `fp` the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The recall is the ratio `tp / (tp + fn)` where `tp` is the number of true positives and `fn` the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [ ]:
target_names = ["No Damage", "Slight Damage", "Moderate Damage", "Heavy Damage", "Very Heavy Damage", "Desctruction"]
print(classification_report(train_labels, in_sample_preds, target_names=target_names))

#### Display the confusion matrix for the predictions on the _training_ set

In [ ]:
confusion_matrix(train_labels, in_sample_preds)

#### Plot confusion matrix with absolute, non-normalized values

In [ ]:
plot_confusion_matrix(
    gs, train_values_subset, train_labels,
    values_format=",",
    cmap="viridis"
)

#### Plot confusion matrix with normalized values

In [ ]:
plot_confusion_matrix(
    gs, train_values_subset, train_labels,
    normalize="true",
    values_format=".2f",
    cmap="cividis"
)

### Partial Dependence and Individual Conditional Expectation

In [ ]:
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence

In [ ]:
tic = time()
numeric_features = ["AGE", "NUM_STOREYS", "MMI", "DPM_MAX"]
categorical_features = ["TYPE_CONSTR", "TYPE_FLOOR", "TYPE_ROOF", "IRREGULARITY"]
display = plot_partial_dependence(
    gs, X_train, numeric_features, target=5, n_cols=2
)
print(f"Plot generated in {time() - tic:.0f}s")
display.figure_.suptitle(
    'Partial dependence of damage grade on non-location features\n'
    'for the California housing dataset, with Gradient Boosting'
)
display.figure_.subplots_adjust(hspace=0.5)

## 9: Make predictions on the test set and evaluate the model performance

### Use the trained ML model to make predictions on the test set

In [ ]:
y_pred = gs.predict(X_test)

### Evaluate the model prediction performance on the _test_ set

#### Display the confusion matrix for the predictions on the _test_ set

In [ ]:
confusion_matrix(y_test, y_pred)

#### Plot confusion matrix with absolute, non-normalized values

In [ ]:
plot_confusion_matrix(
    gs, X_test, y_test,
    values_format=",",
    cmap="viridis"
)

#### Plot confusion matrix with normalized values

In [ ]:
plot_confusion_matrix(
    gs, X_test, y_test,
    normalize="true",
    values_format=".2f",
    cmap="cividis"
)

#### Print classification report for the predictions on the _test_ set

In [ ]:
target_names = ["No Damage", "Slight Damage", "Moderate Damage", "Heavy Damage", "Very Heavy Damage", "Desctruction"]
print(classification_report(y_test, y_pred, target_names=target_names))

#### Print balanced accuracy score for the predictions on the _test_ set

In [ ]:
balanced_accuracy_score(y_test, y_pred)

## 10: Try binary classification

### Compress the damage grades into 0 and 1

In [ ]:
# Assign buildings with no DPM value to a value of DPM=0
inputs_gdf["DAMAGE_GRADE_BINARY"] = inputs_gdf.DAMAGE_GRADE.apply(lambda x: 0 if x<=2 else 1)

### Extract the training features and training labels

In [ ]:
train_values = inputs_gdf[["AGE", "FOOTPRINT_AREA", "NUM_STOREYS", "SLOPE_GROUND", "TYPE_CONSTR", "TYPE_FLOOR", "TYPE_ROOF", "ADJACENCY", "FOOTPRINT_SHAPE", "IRREGULARITY", "MMI", "DPM_MAX"]]
train_labels = inputs_gdf[["DAMAGE_GRADE_BINARY"]]

In [ ]:
train_values.dtypes

In [ ]:
train_labels.dtypes

### Explore the training data

In [ ]:
ax = (train_labels.DAMAGE_GRADE_BINARY
      .value_counts()
      .sort_index()
      .plot.bar(
          title="Number of Buildings in Each Damage Grade",
          color=["Green", "Orange"],
      ))
for p in ax.patches:
    ax.annotate(
        p.get_height(), (p.get_x()+p.get_width()/2., p.get_height()), 
        ha='center', va='center', 
        xytext=(0, 10), textcoords='offset points')

### Select subset of features to use for training

In [ ]:
selected_features = ["AGE", "NUM_STOREYS", "SLOPE_GROUND", "TYPE_CONSTR", "TYPE_FLOOR", "TYPE_ROOF", "ADJACENCY", "FOOTPRINT_SHAPE", "IRREGULARITY", "MMI", "DPM_MAX"]
train_values_subset = train_values[selected_features]

### Explore the relationships between the numeric features and labels

In [ ]:
sns.set_theme(style="ticks")
train_values_subset_plot = train_values_subset.replace(0, np.nan)
sns.pairplot(
    train_values_subset_plot.join(train_labels),
    hue="DAMAGE_GRADE_BINARY",
    hue_order=[1, 0],
    palette={0:"Green", 1:"Orange"},
    markers=["1", "2"],
    kind="scatter",
    diag_kind="hist",
    corner=True,
    plot_kws={'alpha':0.6, 'linewidth':1},
    diag_kws={'fill':True},
    grid_kws={'diag_sharey':False},
)

### Split dataset into training and test subsets

When performing a (supervised) machine learning experiment, it is common to hold out part of the available data as a test set `X_test, y_test`. The best parameters can be determined by grid search techniques.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_values_subset, train_labels, train_size=0.70, test_size=0.30, random_state=40)

In [ ]:
ordinal_encoder = make_column_transformer(
    (OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan),
     make_column_selector(dtype_include='object')),
    remainder='passthrough')

In [ ]:
# The ordinal encoder will first output the categorical features, and then the
# continuous (passed-through) features
categorical_mask = ([True] * n_categorical_features + [False] * n_numerical_features)

In [ ]:
sample_weight = compute_sample_weight(class_weight="balanced", y=y_train)

### Use a Histogram Gradient Boosting Classifier

In [ ]:
pipe = make_pipeline(
    ordinal_encoder,
    HistGradientBoostingClassifier(
        random_state=42,
        categorical_features=categorical_mask)
)
pipe

### Train a HistGradientBoostingClassifier model with scikit-learn defaults for all parameters

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

In [ ]:
gs_binary = pipe
gs_binary.fit(X_train, y_train.values.ravel(), histgradientboostingclassifier__sample_weight=sample_weight)

### Evaluate the model prediction performance on the _training set_

In [ ]:
in_sample_preds = gs_binary.predict(train_values_subset)

#### Plot confusion matrix with absolute, non-normalized values

In [ ]:
plot_confusion_matrix(
    gs_binary, train_values_subset, train_labels,
    values_format=",",
    cmap="viridis"
)

#### Plot confusion matrix with normalized values

In [ ]:
plot_confusion_matrix(
    gs_binary, train_values_subset, train_labels,
    normalize="true",
    values_format=".2f",
    cmap="cividis"
)

### Use the trained model to make predictions on the test set

In [ ]:
y_pred = gs_binary.predict(X_test)

#### Display the confusion matrix for the predictions on the test set

In [ ]:
confusion_matrix(y_test, y_pred)

#### Plot confusion matrix with absolute, non-normalized values

In [ ]:
plot_confusion_matrix(
    gs_binary, X_test, y_test,
    values_format=",",
    cmap="viridis"
)

#### Plot confusion matrix with normalized values

In [ ]:
plot_confusion_matrix(
    gs_binary, X_test, y_test,
    normalize="true",
    values_format=".2f",
    cmap="cividis"
)

#### Print classification report for the predictions on the _test_ set

In [ ]:
target_names = ["No Damage", "Damage"]
print(classification_report(y_test, y_pred, target_names=target_names))

#### Print balanced accuracy score for the predictions on the _test_ set

In [ ]:
balanced_accuracy_score(y_test, y_pred)